In [4]:
source_dir = "src"
!mkdir $source_dir

mkdir: cannot create directory ‘src’: File exists


In [6]:
%%writefile $source_dir/collision.h
#ifndef _COLLISION_H
#define _COLLISION_H

#include <vector>

using namespace std;

class Collision {
  /* A collision event of two bodies of zero radius,
    records the time to collision, location of collision
     and after collision velocities.
   This object is needed because C++ functions cannot
    return more than one values. */
  public:
  Collision(){collision_pos.resize(3);new_vel1.resize(3);new_vel2.resize(3);};
  float time_to_collision;
  vector<float> collision_pos,new_vel1,new_vel2;
};
#endif

Overwriting src/collision.h


In [7]:
%%writefile $source_dir/particle.h
#ifndef _PARTICLE_H
#define _PARTICLE_H

#include <vector>

using namespace std;

class Particle {
    
    vector<float> pos;
    vector<float> vel;
    float mass;
    
    public:
    
    Particle(){mass=1.;pos.resize(3);vel.resize(3);};
    
    vector<float> get_pos(){return this->pos;};
    vector<float> get_vel(){return this->vel;};
    void change_pos(vector<float> pos){this->pos=pos;};
    void change_pos_component(int idx, float value){this->pos[idx]=value;};
    void change_vel(vector<float> vel){this->vel=vel;};
    void change_vel_component(int idx, float value){this->vel[idx]=value;};
    
};
# endif

Overwriting src/particle.h


In [8]:
%%writefile $source_dir/box.h
#ifndef _BOX_H
#define _BOX_H

#include "particle.h"
#include "collision.h"

class Box {
    // cubic box with one cornor at the origin
    vector<float> side_length;
    public:
    Box(float lx,float ly,float lz){
        side_length.resize(3);
        side_length[0] = lx;
        side_length[1] = ly;
        side_length[2] = lz;
    };
    float surface_area();
    
    Collision next_collision_with(Particle* ptcl);
    
};
#endif

Overwriting src/box.h


In [9]:
%%writefile $source_dir/box.cpp

#include "box.h"

float Box::surface_area(){
  return 2.*(side_length[0]*side_length[1] 
           + side_length[1]*side_length[2] 
           + side_length[2]*side_length[0] );
}

Collision Box::next_collision_with(Particle* ptcl){
            
  // record the collision event
  Collision collision;
  collision.collision_pos = ptcl->get_pos();
  
  // calculate what happens after the collision
  collision.new_vel1 = ptcl->get_vel();
  int face_to_collide_with = 0;
  float time_to_collision = 1.0/0.0; // !! better define inf
            
  // loop through each wall, find time to collision
  for (unsigned int dir=0;dir<3;dir++){ // x,y,z
    for (unsigned int plus=0;plus<2;plus++){ // -,+ direction
      
      float distance_to_wall = 0;
      // [position of wall - position of particle] in given direction
      if (plus){
        distance_to_wall = side_length[dir]-ptcl->get_pos()[dir];
      } else {
        distance_to_wall = 0.0 - ptcl->get_pos()[dir];
      }
                                            
      float time = time_to_collision;
      if (distance_to_wall!=0){
        time = distance_to_wall/ptcl->get_vel()[dir];        
      }
      
      // if time is properly calculated this part is fine 
      //  assign nexy collision
      if (time >0 && time < time_to_collision){
        // enumerate faces (-x,+x,-y,+y,-z,+z) -> (0,1,2,3,4,5)
        face_to_collide_with = dir*2+plus;
        time_to_collision = time;
      }
                                            
    }
  }

  collision.time_to_collision = time_to_collision;
  for (unsigned int dir=0;dir<3;dir++){ // x,y,z
    for (unsigned int plus=0;plus<2;plus++){ // -,+ direction
      if (dir*2+plus==face_to_collide_with){
        collision.new_vel1[dir] = -1.*ptcl->get_vel()[dir];      
      }
    }
  }

  return collision;
}

Overwriting src/box.cpp


In [12]:
%%writefile $source_dir/main.cpp
#include <iostream>
#include <numeric> // inner_product
#include <cmath> // sqrt

#include "particle.h"
#include "box.h"

int main(){
    
    // initialize unit volume box
    Box box(1,1,1);
    
    // initialize particle at center of box
    Particle ptcl;
    vector<float> pos0 = {0.5,0.5,0.5};
    ptcl.change_pos(pos0);
    
    // give the particle a push
    ptcl.change_vel_component(0,0.5);
    ptcl.change_vel_component(1,0.2);
    ptcl.change_vel_component(2,-0.3);
    
    // simulate some number of collision events, record total impulse
    //  deposited onto the surface of the box
    
    float t = 0.0;
    float T = 36000.0;
    int max_event = 1;
    
    vector<double> dvel(3);
    float impulse = 0.0;
    
    unsigned int ievent = 0;
    for (ievent = 0; ievent < max_event; ievent += 1){
        
        // find next collision event
        Collision col = box.next_collision_with(&ptcl);
        
        // calculate impulse
        for (unsigned int dir=0;dir<3;dir++){
            dvel[dir] = col.new_vel1[dir] - ptcl.get_vel()[dir];
            float dv2 = inner_product(dvel.begin(),dvel.end(),dvel.begin(),0);
            impulse += sqrt(dv2);
        }
        
        // let particle know of its collision
        ptcl.change_pos(col.collision_pos);
        ptcl.change_vel(col.new_vel1);
        
        // increment time
        t += col.time_to_collision;
        if (t>T) break;
    }
    cout << ievent << endl;
    cout << impulse/t/box.surface_area() << endl;
    
    return 0;
}

Overwriting src/main.cpp


In [13]:
!g++ -std=c++11 -g $source_dir/main.cpp $source_dir/box.cpp -o main
!./main

1
0.5
